In [1]:
# MVP

In [2]:
# import bibliotecas

from selenium import webdriver
from selenium.common.exceptions import *
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException


from unidecode import unidecode
from time import sleep
from datetime import datetime
from datetime import date
import os

import sqlite3

In [3]:
# estabelece conexão com banco de dados

con = sqlite3.connect("botbitarb.db")
cur = con.cursor()

In [4]:
# configurações webdriver

chrome_options = Options()
chrome_options.add_experimental_option(
    'excludeSwitches', ['enable-logging'])
chrome_options.add_argument('--lang=pt-BR')
chrome_options.add_argument('--disable-notifications')

In [5]:
# funções e variáveis auxiliares

def preco_fmt(preco):
    preco = float(preco.replace('R','').replace('$','').replace('.','').replace(',','.').lstrip())
    return preco

timestamp = datetime.now()

In [6]:
# bot

class bitArbBot:

    def push(self):
        self.bot()

    def bot(self):
        
        valorBase = '49,60' # BRL
        
        # login
        self.driver = webdriver.Chrome('chromedriver', options=chrome_options)
        self.link = 'https://market.bitpreco.com/login/?'
        self.driver.get(self.link)
        sleep(5)
        self.driver.find_element(By.ID, "emailfield").send_keys("")
        self.driver.find_element(By.NAME, "pass").send_keys("Falabella@23")
        sleep(3)
        self.driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div/main/div[2]/form/div[4]/button').click()
        sleep(5)
        self.driver.find_element(By.XPATH, '/html/body').send_keys(Keys.ESCAPE)
        sleep(3)
        
        # loop
        while True:
            self.link = 'https://market.bitpreco.com/trade/BTC-BRL'
            self.driver.get(self.link)
            sleep(5)
            self.driver.find_element(By.XPATH, '/html/body').send_keys(Keys.ESCAPE)
            sleep(3)
            # compra
            self.driver.find_element(By.XPATH,
                                     '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[1]/div[2]/div/div/div/button[1]/span[1]/span/span').click()
            # valor base 
            self.driver.find_element(By.XPATH, 
                                     '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[1]/div[2]/div/input').send_keys(valorBase)
            # pega melhor preço de compra unitário em float
            precoCompraTemp = self.driver.find_element(By.XPATH,
                                                       '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[2]/div/div/input').get_attribute('value')
            precoCompra = preco_fmt(precoCompraTemp)
            # efetua ordem de compra
            self.driver.find_element(By.XPATH,
                                     '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[2]/button/span[1]').click()
            # imprime detalhes da transação de compra
            print('--- ordem executada ---')
            print('tipo: compra')
            print(f'preço de compra: {precoCompra}')
            print(f'valor base BRL {valorBase}')
            print(f'dia e hora: {timestamp}')
            # registra transação no banco de dados
            data = [
                ("compra", precoCompra, "-", timestamp),
            ]
            cur.executemany("INSERT INTO transactions VALUES(?, ?, ?, ?)", data)
            con.commit()
            data.clear()

            # venda
            self.driver.find_element(By.XPATH,
                                     '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[1]/div[2]/div/div/div/button[2]/span[1]/span/span').click()
            sleep(5)
            # 100% saldo Bitcoin
            self.driver.find_element(By.XPATH,
                                     '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[1]/div[1]/div/div/div/div/p[4]/span[1]').click()

            # efetua ordem de venda condicionada a ganho em relação ao valor de compra
            tentativas = 0
            vendaExec = False
            while vendaExec == False:
                try:
                    # pega melhor preço de venda unitário em float
                    precoVendaTemp = self.driver.find_element(By.XPATH,
                                                              '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[2]/div/div/input').get_attribute('value')
                    precoVenda = preco_fmt(precoVendaTemp)
                    # efetua ordem de venda condicionada a ganho
                    sleep(10)
                    if precoCompra < precoVenda:
                        self.driver.find_element(By.XPATH,
                                                 '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[2]/button/span[1]').click()
                        rentab = ((precoVenda / precoCompra)-1)*100
                        # imprime detalhes da transação de venda
                        print('--- ordem executada ---')
                        print('tipo: venda')
                        print(f'preço de venda: {precoVenda}')
                        print(f'rentabilidade: {rentab}%')
                        print(f'dia e hora: {timestamp}')
                        # registra transação no banco de dados
                        data = [
                            ("venda", precoVenda, rentab, timestamp),
                        ]
                        cur.executemany("INSERT INTO transactions VALUES(?, ?, ?, ?)", data)
                        con.commit()
                        data.clear()
                        
                        vendaExec = True
                    else:
                        tentativas += 1
                        print(f'tentativa de venda {tentativas}')
                        print('atualizando preço de venda...')
                        sleep(20)
                except NoSuchElementException:
                    #self.link = 'https://market.bitpreco.com/trade/BTC-BRL'
                    #self.driver.get(self.link)
                    #sleep(5)
                    continue



In [ ]:
start = bitArbBot()
start.push()

--- ordem executada ---
tipo: compra
preço de compra: 100492.0
valor base BRL 49,60
dia e hora: 2022-10-11 12:37:40.075415
tentativa de venda 1
atualizando preço de venda...
tentativa de venda 2
atualizando preço de venda...
